In [174]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import gzip
import math as m

In [175]:
stations_file = "stations.json.gz"
cities_file = "city_data.csv"
output_file = "output.svg"

In [176]:
station_fh = gzip.open(stations_file, 'rt', encoding='utf-8')
stations_df = pd.read_json(station_fh, lines=True)
cities_df = pd.read_csv(cities_file)

stations_df['avg_tmax'] = stations_df['avg_tmax'] / 10 #divide by ten weather data is °C×10 (because that's what GHCN distributes)
#the average daily-high temperature for the year

cities_df = cities_df[np.isfinite(cities_df.population)]
cities_df = cities_df[np.isfinite(cities_df.area)].reset_index(drop=True) #get ride of unusable data
cities_df['area'] = cities_df['area'] / 1000000 #convert from m^2 to km^2

cities_df = cities_df[cities_df.area <= 10000] #exclude unreasonable area

cities_df.reset_index(drop=True)

cities_df['density'] = cities_df['population'] / cities_df['area'] #calculate density

In [262]:
def distance(city, stations): #takes the current row of the cities df and the stations df as argument
    #returns a df of distances from the city to all the stations
    p = float(m.pi/180)
    city_lat = city['latitude']
    city_long = city['longitude']

    #row is the city columns are stations
    d = 0.5 - np.cos((stations['latitude']-city_lat)*p)/2 + np.cos(city_lat*p) * np.cos(stations['latitude']*p) * (1- np.cos((stations['longitude']-city_long)*p))/2
    
    return 12742*np.arcsin(np.sqrt(d))

In [264]:
def best_tmax(city, stations): 
    stations['distance'] = distance(city, stations)
    station = stations_df[stations_df['distance'] == stations_df['distance'].min()]
    
    station = station.reset_index(drop=True)
    
    return station.loc[0, 'avg_tmax']

In [269]:
cities_df['avg_tmax'] = cities_df.apply(best_tmax, axis=1, stations=stations_df)
cities_df

,name,population,area,latitude,longitude,density,avg_tmax
0,Calgary,1096833.0,825.290000,51.054444,-114.066944,1329.027372,12.152329
1,Edmonton,812201.0,684.370000,53.500000,-113.500000,1186.786387,11.098338
2,Abbotsford,133497.0,375.550000,49.054611,-122.328000,355.470643,15.782787
3,Burnaby,223218.0,90.610000,49.250000,-122.949167,2463.502925,13.876667
4,Nanaimo,83811.0,91.000000,49.164167,-123.936389,921.000000,13.864754
5,Revelstoke,7139.0,40.000000,50.998100,-118.196000,178.475000,13.483978
6,Richmond,198309.0,129.270000,49.166667,-123.133333,1534.068229,16.176136
7,Surrey,468251.0,316.410000,49.183300,-122.850000,1479.886856,13.439610
8,Vancouver,603502.0,115.000000,49.250000,-123.100000,5247.843478,15.153846
9,Victoria,80032.0,19.470000,48.422151,-123.365700,4110.529019,15.129146


In [268]:
'''city = cities_df.loc[0]
p = float(m.pi/180)
city_lat = city['latitude']
city_long = city['longitude']
    
#row is the city columns are stations
d = 0.5 - np.cos((stations_df['latitude']-city_lat)*p)/2 + np.cos(city_lat*p) * np.cos(stations_df['latitude']*p) * (1- np.cos((stations_df['longitude']-city_long)*p))/2
stations_df['distance'] = 12742*np.arcsin(np.sqrt(d))
station = stations_df[stations_df['distance'] == stations_df['distance'].min()]
station = station.reset_index(drop=True)
station.loc[0, 'avg_tmax']'''

#code to test functionality



"city = cities_df.loc[0]\np = float(m.pi/180)\ncity_lat = city['latitude']\ncity_long = city['longitude']\n    \n#row is the city columns are stations\nd = 0.5 - np.cos((stations_df['latitude']-city_lat)*p)/2 + np.cos(city_lat*p) * np.cos(stations_df['latitude']*p) * (1- np.cos((stations_df['longitude']-city_long)*p))/2\nstations_df['distance'] = 12742*np.arcsin(np.sqrt(d))\nstation = stations_df[stations_df['distance'] == stations_df['distance'].min()]\nstation = station.reset_index(drop=True)\nstation.loc[0, 'avg_tmax']"

In [276]:
plt.scatter(cities_df['avg_tmax'], cities_df['population'])

In [275]:
plt.savefig('output.svg')